In [1]:
import warnings
import os
import argparse
from numpy.lib.function_base import average
import pandas as pd
warnings.filterwarnings('ignore')
import numpy as np


from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import make_scorer, roc_auc_score
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

from train_val import *

In [2]:
base_path = "/media/z/data2/artery_semantic_segmentation/ml"
data_path = "data/feature.csv"
feature_df = pd.read_csv(os.path.join(base_path, data_path))

In [3]:
LABEL_MAP = {"LMA": 0, "LAD1": 1, "LAD2": 1, "LAD3": 1, 
             "LCX1": 2, "LCX2": 2, "LCX3": 2, 
             "D1": 3, "D2": 3, "OM1": 4, "OM2":4}

In [4]:
labels = [LABEL_MAP[k] for k in feature_df['label']]
feature_df["class"] = labels

In [5]:
feature_df

,patient_id,label,view,n_centerline,n_pixels,original_firstorder_10Percentile,original_firstorder_90Percentile,original_firstorder_Energy,original_firstorder_Entropy,original_firstorder_InterquartileRange,...,p2_y_weighted_center,r_max,r_mean,r_min,r_std,weighted_x_center,weighted_y_center,x_center,y_center,class
0,B01_LCA_LAO,LMA,1,0.025253,0.019045,0.714286,0.519737,0.028082,0.000000,0.101266,...,0.429497,0.235143,0.209159,0.200000,0.187715,0.089256,0.473606,0.092913,0.455219,0
1,B01_LCA_LAO,LCX1,1,0.045455,0.042419,0.669173,0.519737,0.049151,0.267718,0.109705,...,0.452504,0.215982,0.237045,0.173703,0.168051,0.126720,0.499336,0.133222,0.480586,2
2,B01_LCA_LAO,LAD1,1,0.069024,0.040317,0.706767,0.517763,0.049113,0.087863,0.084388,...,0.431919,0.182635,0.171606,0.133333,0.099302,0.141839,0.485236,0.147798,0.466799,1
3,B01_LCA_LAO,D1,1,0.469697,0.216300,0.733835,0.578947,0.258566,0.235154,0.135021,...,0.450082,0.154775,0.096275,0.000000,0.217901,0.398416,0.477892,0.421047,0.458941,3
4,B01_LCA_LAO,LAD2,1,0.749158,0.395004,0.669173,0.539474,0.404937,0.312158,0.168776,...,0.280554,0.182635,0.125296,0.000000,0.207063,0.496231,0.298058,0.520190,0.281339,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,210_LCA_RAO,LCX2,0,0.632997,0.328098,0.661654,0.506579,0.322719,0.324655,0.101266,...,0.672923,0.235143,0.153360,0.000000,0.342104,0.737767,0.438470,0.765843,0.427130,2
2306,210_LCA_RAO,LMA,0,0.111111,0.161514,0.548872,0.447368,0.136241,0.371703,0.168776,...,0.159427,0.399546,0.498514,0.441052,0.138989,0.190960,0.216102,0.200536,0.209695,0
2307,210_LCA_RAO,LAD1,0,0.360269,0.335147,0.699248,0.684211,0.446534,0.549350,0.354430,...,0.617453,0.302769,0.304738,0.200000,0.226950,0.170443,0.524396,0.177684,0.515208,1
2308,210_LCA_RAO,D1,0,0.271044,0.147663,0.872932,0.736842,0.253964,0.337854,0.185654,...,0.764845,0.149830,0.128226,0.000000,0.209725,0.503385,0.825015,0.528788,0.825464,3


In [6]:
all_subjects = feature_df["patient_id"].unique()

In [7]:
train_subjects, test_subjects = get_split_deterministic(all_subjects, 0, 5)

In [8]:
train_df = feature_df[feature_df['patient_id'].isin(train_subjects)]
test_df = feature_df[feature_df['patient_id'].isin(test_subjects)]

train_y = train_df['class'].to_numpy()
train_df = train_df.drop(columns=["patient_id", "label", "class"])
train_x = train_df.to_numpy()

test_y = test_df['class'].to_numpy()
test_df = test_df.drop(columns=["patient_id", "label", "class"])
test_x = test_df.to_numpy()

train_df.shape
test_df.shape

(467, 122)

In [9]:
train_lr(train_x, train_y, test_x, test_y, n_split=5)

LogisticRegression(C=100, random_state=42)
{'C': 100, 'penalty': 'l2'}
[x] performance for LR classifier
Training:
accuracy:0.273
recall:0.273
precision:0.210
 
Test:
accuracy:0.255
recall:0.255
precision:0.191
 


(LogisticRegression(C=100, random_state=42), {'C': 100, 'penalty': 'l2'})

In [10]:
train_knn(train_x, train_y, test_x, test_y, n_split=5)

KNeighborsClassifier(n_neighbors=6)
{'n_neighbors': 6, 'weights': 'uniform'}
[x] performance for KNN classifier
Training:
accuracy:0.270
recall:0.270
precision:0.240
 
Test:
accuracy:0.259
recall:0.259
precision:0.223
 


(KNeighborsClassifier(n_neighbors=6), {'n_neighbors': 6, 'weights': 'uniform'})

In [11]:
train_rf(train_x, train_y, test_x, test_y, n_split=5)

/home/z/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/z/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/home/z/.local/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for 

KeyboardInterrupt: 

In [12]:
train_sgd(train_x, train_y, test_x, test_y, n_split=5)

SGDClassifier(alpha=0.001, loss='log', max_iter=100, penalty='l1',
              random_state=42)
{'alpha': 0.001, 'max_iter': 100, 'penalty': 'l1'}
[x] performance for SGD classifier
Training:
accuracy:0.226
recall:0.226
precision:0.212
 
Test:
accuracy:0.216
recall:0.216
precision:0.198
 


(SGDClassifier(alpha=0.001, loss='log', max_iter=100, penalty='l1',
               random_state=42),
 {'alpha': 0.001, 'max_iter': 100, 'penalty': 'l1'})

In [13]:
train_svm(train_x, train_y, test_x, test_y, n_split=5)

SVC(C=1000, gamma=0.001, probability=True)
{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
[x] performance for SVM classifier
Training:
accuracy:0.277
recall:0.277
precision:0.215
 
Test:
accuracy:0.261
recall:0.261
precision:0.202
 


(SVC(C=1000, gamma=0.001, probability=True),
 {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'})

In [23]:
from sklearn import metrics

MAIN_BRANCH_CATEGORY = ["LMA", "LAD", "LCX", "D", "OM"]

branches = ["LMA", "LAD", "LCX", "D", "OM", "weighted avg"]
acc, prec, rec, f1 = {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}, {k: [] for k in branches}

for cv in range(5):
    svc = SVC(probability=True, C=1000, gamma=0.001, kernel='rbf')
    train_idx, test_idx = get_split_deterministic(list(range(len(train_x))), cv, 5)
    train_x_cv, train_y_cv = train_x[train_idx], train_y[train_idx]
    svc.fit(train_x_cv, train_y_cv)
    pred_y = svc.predict(test_x)

    cm = metrics.confusion_matrix(test_y, pred_y)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    # acc = metrics.accuracy_score(test_y, pred_y)
    # precision = metrics.precision_score(test_y, pred_y, average="weighted")
    # recall = metrics.recall_score(test_y, pred_y, average="weighted")
    # f1_score = metrics.f1_score(test_y, pred_y, average="weighted")

    clf_report = metrics.classification_report(test_y, pred_y, target_names=MAIN_BRANCH_CATEGORY, output_dict=True)

    for idx, k in enumerate(branches):
        if idx < len(branches) - 1:
            acc[k].append(cm.diagonal()[idx])
        else:
            acc[k].append(clf_report["accuracy"])
        prec[k].append(clf_report[k]['precision'])
        rec[k].append(clf_report[k]['recall'])
        f1[k].append(clf_report[k]['f1-score'])

In [29]:
table_df = pd.DataFrame(columns=["metric"]+branches)
row_data = {"metric": "acc"}
for k in branches:
    row_data.update({k: f"{np.mean(acc[k]):0.4f}±{np.std(acc[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
row_data = {"metric": "pre"}
for k in branches:
    row_data.update({k: f"{np.mean(prec[k]):0.4f}±{np.std(prec[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
row_data = {"metric": "rec"}
for k in branches:
    row_data.update({k: f"{np.mean(rec[k]):0.4f}±{np.std(rec[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
row_data = {"metric": "f1"}
for k in branches:
    row_data.update({k: f"{np.mean(f1[k]):0.4f}±{np.std(f1[k]):0.4f}"})
table_df = table_df.append(row_data, ignore_index=True)
table_df.to_csv("ml.csv", index=False)

In [ ]:
train_gradient_boosting(train_x, train_y, test_x, test_y, n_split=5)

KeyboardInterrupt: 